# IQ Imbalance Transform
I/Q Imbalance is a predominantly radio hardware effect that comes from direct conversion transmitters and receivers whose I and Q channels are not perfectly $90^o$ aligned with one another. A relative phase error between the I and Q channels will cause deterministic phase and amplitude distortion, alongside a direct-current (DC) offset.

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# TorchSig
import torchsig.transforms.functional as F
from torchsig.signals.signal_types import Signal
from torchsig.utils.defaults import default_dataset

The following function creates a test QPSK signal.

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

IQ imbalance is applied to the QPSK signal. The effect manifests itself by distorting the otherwise square constellation shape and marginally increasing the DC offset by moving constellation the direction represented by `dc_offset_phase_rads`.

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_signal(num_iq_samples=N, scale=1.0).data

# IQ imbalance model
impaired_qpsk_data = F.iq_imbalance(
    data=qpsk_data,
    amplitude_imbalance=1,
    phase_imbalance=np.pi / 10,
    dc_offset_db=-20.0,
    dc_offset_phase_rads=-np.pi / 20,
    noise_power_db=0.0,
)  # dB

freq_vec = np.arange(-1.0 / 2, 1.0 / 2, 1.0 / (N))
qpsk_fft = np.fft.fftshift(np.fft.fft(qpsk_data)) / np.sqrt(N)
impaired_qpsk_fft = np.fft.fftshift(np.fft.fft(impaired_qpsk_data)) / np.sqrt(N)

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(2, 1, 1)
ax.plot(freq_vec, 20 * np.log10(np.abs(qpsk_fft)), label="Input")
ax.plot(
    freq_vec, 20 * np.log10(np.abs(impaired_qpsk_fft)), alpha=0.5, label="IQ Imbalance"
)
ax.set_ylim([-70, 0])
ax.set_xlim([freq_vec[0], freq_vec[-1]])
ax.set_ylabel("Magnitude (log10)", fontsize="large")
ax.set_title("QPSK with/without IQ Imbalance Impairment")
ax.legend(fontsize="large", loc="upper left")
ax.grid()

ax = fig.add_subplot(2, 1, 2)
ax.set_box_aspect(1)
ax.plot(np.real(qpsk_data), np.imag(qpsk_data), label="Input")
ax.plot(
    np.real(impaired_qpsk_data),
    np.imag(impaired_qpsk_data),
    alpha=0.5,
    label="IQ Imbalance",
)
ax.grid()
ax.set_title("Constellation Plot")
ax.set_xlabel("Real")
ax.set_ylabel("Imag")
ax.legend(fontsize="large", loc="upper left");